# 🔄 Continuous Learning & Model Updates: Practical Implementation

## Table of Contents
1. [Setup and Data Preparation](#practice-1-setup-and-data-preparation)
2. [Concept Drift Detection](#practice-2-concept-drift-detection)
3. [Catastrophic Forgetting Prevention](#practice-3-catastrophic-forgetting-prevention)
4. [Memory Replay Strategy](#practice-4-memory-replay-strategy)
5. [Model Update Pipeline](#practice-5-model-update-pipeline)
6. [Performance Monitoring](#practice-6-performance-monitoring)
7. [Rollback Mechanism](#practice-7-rollback-mechanism)

## Installing and Importing Essential Libraries

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')

print("✅ All libraries loaded successfully!")
print("📚 Ready for Continuous Learning practice!")

---
## Practice 1: Setup and Data Preparation

### 🎯 Learning Objectives
- Simulate evolving medical data streams
- Create initial and drift datasets
- Understand data distribution changes

### 📖 Key Concepts
**Concept Drift**: Changes in data distribution over time that affect model performance

In [ ]:
# 1.1 Generate initial medical dataset (e.g., disease diagnosis)
def create_initial_dataset(n_samples=1000, random_state=42):
    """
    Create initial medical dataset
    Features: Patient vitals, lab results
    Target: Disease presence (0/1)
    """
    X, y = make_classification(
        n_samples=n_samples,
        n_features=10,
        n_informative=7,
        n_redundant=2,
        n_classes=2,
        weights=[0.7, 0.3],  # Imbalanced: 70% healthy, 30% disease
        flip_y=0.05,  # 5% label noise
        random_state=random_state
    )
    
    # Create meaningful feature names
    feature_names = [
        'heart_rate', 'blood_pressure', 'temperature', 'oxygen_sat',
        'glucose', 'white_blood_cells', 'red_blood_cells', 'hemoglobin',
        'age', 'bmi'
    ]
    
    df = pd.DataFrame(X, columns=feature_names)
    df['diagnosis'] = y
    
    print("📊 Initial Dataset Created")
    print(f"   Shape: {df.shape}")
    print(f"   Class distribution: {df['diagnosis'].value_counts().to_dict()}")
    print(f"   Features: {', '.join(feature_names[:5])}...")
    
    return df, feature_names

# Create initial dataset
initial_data, feature_names = create_initial_dataset()
initial_data.head()

In [ ]:
# 1.2 Generate drift dataset (simulating disease evolution or new variant)
def create_drift_dataset(n_samples=500, drift_strength=0.5, random_state=100):
    """
    Create dataset with concept drift
    Simulates: new disease variant, population change, seasonal effects
    """
    X, y = make_classification(
        n_samples=n_samples,
        n_features=10,
        n_informative=7,
        n_redundant=2,
        n_classes=2,
        weights=[0.5, 0.5],  # More balanced distribution (drift!)
        flip_y=0.1,  # Increased noise
        random_state=random_state
    )
    
    # Add drift by shifting features
    X = X + drift_strength * np.random.randn(*X.shape)
    
    df = pd.DataFrame(X, columns=feature_names)
    df['diagnosis'] = y
    
    print("\n⚠️ Drift Dataset Created")
    print(f"   Shape: {df.shape}")
    print(f"   Class distribution: {df['diagnosis'].value_counts().to_dict()}")
    print(f"   Drift strength: {drift_strength}")
    
    return df

# Create drift dataset
drift_data = create_drift_dataset()
drift_data.head()

---
## Practice 2: Concept Drift Detection

### 🎯 Learning Objectives
- Detect distribution changes using statistical tests
- Implement KS test for drift detection
- Monitor model performance degradation

### 📖 Key Concepts
**KS Test**: Kolmogorov-Smirnov test measures the difference between two distributions

In [ ]:
# 2.1 Implement drift detection using KS test
def detect_drift_ks_test(data_old, data_new, feature_names, threshold=0.05):
    """
    Detect concept drift using Kolmogorov-Smirnov test
    
    Returns:
        drift_detected: Boolean
        drift_features: List of features with significant drift
    """
    drift_features = []
    p_values = {}
    
    print("🔍 Running Drift Detection (KS Test)")
    print("=" * 60)
    
    for feature in feature_names:
        # Perform KS test
        statistic, p_value = stats.ks_2samp(
            data_old[feature], 
            data_new[feature]
        )
        
        p_values[feature] = p_value
        
        # Check if drift detected
        if p_value < threshold:
            drift_features.append(feature)
            status = "⚠️ DRIFT"
        else:
            status = "✅ OK"
        
        print(f"{feature:20s}: p-value={p_value:.4f} {status}")
    
    drift_detected = len(drift_features) > 0
    
    print("\n" + "=" * 60)
    print(f"Drift detected: {drift_detected}")
    print(f"Features with drift: {len(drift_features)}/{len(feature_names)}")
    if drift_features:
        print(f"Affected features: {', '.join(drift_features[:5])}...")
    
    return drift_detected, drift_features, p_values

# Run drift detection
drift_detected, drift_features, p_values = detect_drift_ks_test(
    initial_data, 
    drift_data, 
    feature_names
)

In [ ]:
# 2.2 Visualize drift detection results
def visualize_drift(data_old, data_new, feature_names, p_values):
    """
    Visualize distribution changes and drift severity
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. P-value bar chart
    ax1 = axes[0, 0]
    features_sorted = sorted(p_values.items(), key=lambda x: x[1])
    features_list = [f[0] for f in features_sorted]
    pvals_list = [f[1] for f in features_sorted]
    
    colors = ['red' if p < 0.05 else 'green' for p in pvals_list]
    ax1.barh(features_list, pvals_list, color=colors, alpha=0.7)
    ax1.axvline(x=0.05, color='red', linestyle='--', label='Threshold (0.05)')
    ax1.set_xlabel('P-value')
    ax1.set_title('Drift Detection: KS Test P-values')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Distribution comparison for most drifted feature
    ax2 = axes[0, 1]
    most_drift_feature = features_sorted[0][0]
    ax2.hist(data_old[most_drift_feature], bins=30, alpha=0.5, label='Original', color='blue')
    ax2.hist(data_new[most_drift_feature], bins=30, alpha=0.5, label='New Data', color='red')
    ax2.set_xlabel('Value')
    ax2.set_ylabel('Frequency')
    ax2.set_title(f'Distribution Shift: {most_drift_feature}')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Class distribution comparison
    ax3 = axes[1, 0]
    class_old = data_old['diagnosis'].value_counts().sort_index()
    class_new = data_new['diagnosis'].value_counts().sort_index()
    x = np.arange(len(class_old))
    width = 0.35
    ax3.bar(x - width/2, class_old.values, width, label='Original', alpha=0.7, color='blue')
    ax3.bar(x + width/2, class_new.values, width, label='New Data', alpha=0.7, color='red')
    ax3.set_xlabel('Class')
    ax3.set_ylabel('Count')
    ax3.set_title('Class Distribution Change')
    ax3.set_xticks(x)
    ax3.set_xticklabels(['Healthy', 'Disease'])
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Feature mean shifts
    ax4 = axes[1, 1]
    mean_shifts = []
    for feature in feature_names:
        shift = abs(data_new[feature].mean() - data_old[feature].mean())
        mean_shifts.append(shift)
    
    ax4.bar(range(len(feature_names)), mean_shifts, alpha=0.7, color='orange')
    ax4.set_xlabel('Feature Index')
    ax4.set_ylabel('Mean Shift (Absolute)')
    ax4.set_title('Feature Mean Shifts')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Visualization complete!")
    print(f"   Most drifted feature: {most_drift_feature}")
    print(f"   Mean shift magnitude: {mean_shifts[0]:.4f}")

# Visualize drift
visualize_drift(initial_data, drift_data, feature_names, p_values)

---
## Practice 3: Catastrophic Forgetting Prevention

### 🎯 Learning Objectives
- Train initial model and measure baseline performance
- Observe catastrophic forgetting when training on new data only
- Compare with and without memory replay

### 📖 Key Concepts
**Catastrophic Forgetting**: Rapid loss of previously learned knowledge when learning new information

In [ ]:
# 3.1 Train initial model
def train_initial_model(data, feature_names):
    """
    Train baseline model on initial dataset
    """
    X = data[feature_names]
    y = data['diagnosis']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Train model
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    
    # Evaluate
    train_acc = accuracy_score(y_train, model.predict(X_train))
    test_acc = accuracy_score(y_test, model.predict(X_test))
    test_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    print("🎯 Initial Model Training Complete")
    print("=" * 60)
    print(f"Training accuracy:   {train_acc:.4f}")
    print(f"Test accuracy:       {test_acc:.4f}")
    print(f"Test AUC:            {test_auc:.4f}")
    print("=" * 60)
    
    return model, X_test, y_test

# Train initial model
initial_model, X_test_initial, y_test_initial = train_initial_model(
    initial_data, 
    feature_names
)

In [ ]:
# 3.2 Demonstrate catastrophic forgetting
def demonstrate_catastrophic_forgetting(initial_model, drift_data, 
                                       X_test_initial, y_test_initial,
                                       feature_names):
    """
    Train on new data only and observe performance drop on old data
    """
    print("⚠️ Demonstrating Catastrophic Forgetting")
    print("=" * 60)
    
    # Performance on initial test set BEFORE retraining
    acc_before = accuracy_score(
        y_test_initial, 
        initial_model.predict(X_test_initial)
    )
    print(f"Performance on initial data BEFORE retraining: {acc_before:.4f}")
    
    # Train ONLY on new drift data (this causes forgetting)
    X_drift = drift_data[feature_names]
    y_drift = drift_data['diagnosis']
    
    new_model = LogisticRegression(random_state=42, max_iter=1000)
    new_model.fit(X_drift, y_drift)
    
    # Performance on initial test set AFTER retraining
    acc_after = accuracy_score(
        y_test_initial, 
        new_model.predict(X_test_initial)
    )
    print(f"Performance on initial data AFTER retraining:  {acc_after:.4f}")
    
    # Calculate forgetting
    forgetting = acc_before - acc_after
    print("\n" + "=" * 60)
    print(f"❌ Catastrophic Forgetting Magnitude: {forgetting:.4f}")
    print(f"   ({abs(forgetting/acc_before)*100:.1f}% performance drop)")
    print("=" * 60)
    
    return new_model, forgetting

# Demonstrate forgetting
forgetting_model, forgetting_magnitude = demonstrate_catastrophic_forgetting(
    initial_model, 
    drift_data,
    X_test_initial, 
    y_test_initial,
    feature_names
)

---
## Practice 4: Memory Replay Strategy

### 🎯 Learning Objectives
- Implement memory replay buffer
- Train model with mixed batches (old + new data)
- Prevent catastrophic forgetting

### 📖 Key Concepts
**Memory Replay**: Store representative samples from old data and replay them during retraining

In [ ]:
# 4.1 Create memory replay buffer
def create_replay_buffer(data, buffer_size=100, sampling_strategy='random'):
    """
    Create memory buffer with representative samples
    
    Strategies:
    - random: Random sampling
    - balanced: Class-balanced sampling
    """
    if sampling_strategy == 'random':
        buffer = data.sample(n=min(buffer_size, len(data)), random_state=42)
    
    elif sampling_strategy == 'balanced':
        # Sample equally from each class
        samples_per_class = buffer_size // 2
        class_0 = data[data['diagnosis'] == 0].sample(
            n=min(samples_per_class, (data['diagnosis'] == 0).sum()), 
            random_state=42
        )
        class_1 = data[data['diagnosis'] == 1].sample(
            n=min(samples_per_class, (data['diagnosis'] == 1).sum()), 
            random_state=42
        )
        buffer = pd.concat([class_0, class_1])
    
    print("💾 Memory Replay Buffer Created")
    print("=" * 60)
    print(f"Buffer size: {len(buffer)}")
    print(f"Strategy: {sampling_strategy}")
    print(f"Class distribution: {buffer['diagnosis'].value_counts().to_dict()}")
    print("=" * 60)
    
    return buffer

# Create replay buffer from initial data
replay_buffer = create_replay_buffer(
    initial_data, 
    buffer_size=100, 
    sampling_strategy='balanced'
)

In [ ]:
# 4.2 Train with memory replay
def train_with_replay(replay_buffer, new_data, feature_names, 
                     X_test_initial, y_test_initial):
    """
    Train model with mixed batch (replay buffer + new data)
    """
    print("🔄 Training with Memory Replay")
    print("=" * 60)
    
    # Combine replay buffer and new data
    combined_data = pd.concat([replay_buffer, new_data])
    print(f"Combined training data size: {len(combined_data)}")
    print(f"  - Replay buffer: {len(replay_buffer)}")
    print(f"  - New data: {len(new_data)}")
    
    # Prepare data
    X_combined = combined_data[feature_names]
    y_combined = combined_data['diagnosis']
    
    # Train model
    replay_model = LogisticRegression(random_state=42, max_iter=1000)
    replay_model.fit(X_combined, y_combined)
    
    # Evaluate on initial test set
    acc_initial = accuracy_score(
        y_test_initial, 
        replay_model.predict(X_test_initial)
    )
    
    # Evaluate on new data
    X_new = new_data[feature_names]
    y_new = new_data['diagnosis']
    X_new_test, y_new_test = X_new[:100], y_new[:100]  # Use subset for testing
    acc_new = accuracy_score(
        y_new_test, 
        replay_model.predict(X_new_test)
    )
    
    print("\n📊 Results:")
    print(f"  Performance on initial data: {acc_initial:.4f}")
    print(f"  Performance on new data:     {acc_new:.4f}")
    print("=" * 60)
    print("✅ Successfully maintained performance on both datasets!")
    
    return replay_model, acc_initial, acc_new

# Train with replay
replay_model, acc_old, acc_new = train_with_replay(
    replay_buffer,
    drift_data,
    feature_names,
    X_test_initial,
    y_test_initial
)

In [ ]:
# 4.3 Compare all approaches
def compare_approaches(initial_model, forgetting_model, replay_model,
                      X_test_initial, y_test_initial, forgetting_magnitude):
    """
    Compare performance of different update strategies
    """
    print("\n📊 Comparison of Update Strategies")
    print("=" * 60)
    
    # Calculate accuracies
    acc_initial = accuracy_score(
        y_test_initial, 
        initial_model.predict(X_test_initial)
    )
    acc_forgetting = accuracy_score(
        y_test_initial, 
        forgetting_model.predict(X_test_initial)
    )
    acc_replay = accuracy_score(
        y_test_initial, 
        replay_model.predict(X_test_initial)
    )
    
    # Create comparison table
    comparison = pd.DataFrame({
        'Approach': ['Initial Model', 'Without Replay', 'With Replay'],
        'Accuracy': [acc_initial, acc_forgetting, acc_replay],
        'Performance Drop': [0, forgetting_magnitude, acc_initial - acc_replay]
    })
    
    print(comparison.to_string(index=False))
    print("=" * 60)
    
    # Visualize comparison
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart of accuracies
    ax1 = axes[0]
    bars = ax1.bar(
        comparison['Approach'], 
        comparison['Accuracy'],
        color=['blue', 'red', 'green'],
        alpha=0.7
    )
    ax1.set_ylabel('Accuracy')
    ax1.set_title('Model Performance Comparison')
    ax1.set_ylim([0, 1])
    ax1.axhline(y=acc_initial, color='blue', linestyle='--', alpha=0.5, label='Baseline')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom')
    
    # Performance drop comparison
    ax2 = axes[1]
    drops = comparison['Performance Drop'].abs()
    bars2 = ax2.bar(
        comparison['Approach'], 
        drops,
        color=['gray', 'red', 'green'],
        alpha=0.7
    )
    ax2.set_ylabel('Performance Drop (Absolute)')
    ax2.set_title('Catastrophic Forgetting Magnitude')
    ax2.grid(True, alpha=0.3)
    
    # Add value labels
    for bar in bars2:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Key Insight:")
    print(f"   Memory Replay reduced forgetting by {(forgetting_magnitude - (acc_initial - acc_replay))/forgetting_magnitude*100:.1f}%")

# Compare approaches
compare_approaches(
    initial_model, 
    forgetting_model, 
    replay_model,
    X_test_initial, 
    y_test_initial,
    forgetting_magnitude
)

---
## Practice 5: Model Update Pipeline

### 🎯 Learning Objectives
- Build automated update pipeline
- Implement validation gates
- Create version control for models

### 📖 Key Concepts
**Update Pipeline**: Automated workflow for detecting drift, retraining, and deploying models

In [ ]:
# 5.1 Complete update pipeline
class ModelUpdatePipeline:
    """
    Automated model update pipeline with drift detection and validation
    """
    
    def __init__(self, feature_names, drift_threshold=0.05, 
                 performance_threshold=0.05, replay_buffer_size=100):
        self.feature_names = feature_names
        self.drift_threshold = drift_threshold
        self.performance_threshold = performance_threshold
        self.replay_buffer_size = replay_buffer_size
        
        self.current_model = None
        self.model_versions = []
        self.replay_buffer = None
        self.baseline_performance = None
        
    def initialize(self, initial_data):
        """Initialize pipeline with baseline model"""
        print("🚀 Initializing Update Pipeline...")
        
        # Train initial model
        X = initial_data[self.feature_names]
        y = initial_data['diagnosis']
        
        self.current_model = LogisticRegression(random_state=42, max_iter=1000)
        self.current_model.fit(X, y)
        
        # Store baseline performance
        self.baseline_performance = accuracy_score(y, self.current_model.predict(X))
        
        # Create replay buffer
        self.replay_buffer = initial_data.sample(
            n=min(self.replay_buffer_size, len(initial_data)), 
            random_state=42
        )
        
        # Save version
        self.model_versions.append({
            'version': 'v1.0',
            'model': self.current_model,
            'performance': self.baseline_performance,
            'timestamp': pd.Timestamp.now()
        })
        
        print(f"✅ Pipeline initialized with baseline accuracy: {self.baseline_performance:.4f}")
        
    def check_update_needed(self, new_data):
        """Check if model update is needed"""
        print("\n🔍 Checking if update needed...")
        
        # 1. Drift detection
        drift_features = []
        for feature in self.feature_names:
            _, p_value = stats.ks_2samp(
                self.replay_buffer[feature],
                new_data[feature]
            )
            if p_value < self.drift_threshold:
                drift_features.append(feature)
        
        drift_detected = len(drift_features) > 0
        
        # 2. Performance check
        X_new = new_data[self.feature_names]
        y_new = new_data['diagnosis']
        current_performance = accuracy_score(y_new, self.current_model.predict(X_new))
        performance_drop = self.baseline_performance - current_performance
        
        update_needed = (drift_detected or 
                        performance_drop > self.performance_threshold)
        
        print(f"   Drift detected: {drift_detected} ({len(drift_features)} features)")
        print(f"   Performance drop: {performance_drop:.4f}")
        print(f"   Update needed: {update_needed}")
        
        return update_needed, drift_features, performance_drop
        
    def update_model(self, new_data):
        """Update model with new data and replay buffer"""
        print("\n🔄 Updating model...")
        
        # Combine data
        combined_data = pd.concat([self.replay_buffer, new_data])
        X_combined = combined_data[self.feature_names]
        y_combined = combined_data['diagnosis']
        
        # Train new model
        new_model = LogisticRegression(random_state=42, max_iter=1000)
        new_model.fit(X_combined, y_combined)
        
        # Validate new model
        new_performance = accuracy_score(y_combined, new_model.predict(X_combined))
        
        print(f"   New model performance: {new_performance:.4f}")
        
        # Only deploy if performance is acceptable
        if new_performance >= self.baseline_performance - 0.1:  # Allow 10% drop
            self.current_model = new_model
            
            # Update replay buffer (add new samples)
            new_samples = new_data.sample(
                n=min(self.replay_buffer_size // 2, len(new_data)), 
                random_state=42
            )
            self.replay_buffer = pd.concat([
                self.replay_buffer.iloc[:self.replay_buffer_size // 2],
                new_samples
            ])
            
            # Save version
            version_num = len(self.model_versions) + 1
            self.model_versions.append({
                'version': f'v{version_num}.0',
                'model': new_model,
                'performance': new_performance,
                'timestamp': pd.Timestamp.now()
            })
            
            print(f"✅ Model updated to version v{version_num}.0")
            return True
        else:
            print("❌ New model performance insufficient. Update rejected.")
            return False
    
    def run_pipeline(self, new_data):
        """Run complete update pipeline"""
        print("\n" + "=" * 60)
        print("🔄 Running Update Pipeline")
        print("=" * 60)
        
        # Check if update needed
        update_needed, drift_features, perf_drop = self.check_update_needed(new_data)
        
        if update_needed:
            # Update model
            success = self.update_model(new_data)
            
            if success:
                print("\n✅ Pipeline execution successful!")
            else:
                print("\n⚠️ Pipeline execution completed with warnings.")
        else:
            print("\n✅ No update needed. Model is performing well.")
        
        print("=" * 60)

# Initialize and run pipeline
pipeline = ModelUpdatePipeline(
    feature_names=feature_names,
    drift_threshold=0.05,
    performance_threshold=0.05,
    replay_buffer_size=100
)

pipeline.initialize(initial_data)
pipeline.run_pipeline(drift_data)

---
## Practice 6: Performance Monitoring

### 🎯 Learning Objectives
- Track model performance over time
- Create monitoring dashboard
- Set up alert thresholds

### 📖 Key Concepts
**Performance Monitoring**: Continuous tracking of model metrics to detect degradation

In [ ]:
# 6.1 Performance monitoring dashboard
def create_monitoring_dashboard(pipeline):
    """
    Create performance monitoring visualization
    """
    print("📊 Creating Performance Monitoring Dashboard...\n")
    
    # Extract version history
    versions = [v['version'] for v in pipeline.model_versions]
    performances = [v['performance'] for v in pipeline.model_versions]
    timestamps = [v['timestamp'] for v in pipeline.model_versions]
    
    # Create figure
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Performance over versions
    ax1 = axes[0, 0]
    ax1.plot(versions, performances, marker='o', linewidth=2, markersize=8, color='blue')
    ax1.axhline(y=pipeline.baseline_performance, color='green', 
                linestyle='--', label='Baseline', alpha=0.7)
    ax1.axhline(y=pipeline.baseline_performance - 0.1, color='red', 
                linestyle='--', label='Threshold (-10%)', alpha=0.7)
    ax1.set_xlabel('Model Version')
    ax1.set_ylabel('Accuracy')
    ax1.set_title('Model Performance Over Versions')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0.5, 1.0])
    
    # 2. Replay buffer composition
    ax2 = axes[0, 1]
    buffer_dist = pipeline.replay_buffer['diagnosis'].value_counts()
    ax2.pie(buffer_dist.values, labels=['Healthy', 'Disease'], 
            autopct='%1.1f%%', startangle=90, colors=['lightblue', 'lightcoral'])
    ax2.set_title('Replay Buffer Class Distribution')
    
    # 3. Feature importance (coefficients)
    ax3 = axes[1, 0]
    coef = pipeline.current_model.coef_[0]
    feature_importance = pd.DataFrame({
        'feature': pipeline.feature_names,
        'importance': np.abs(coef)
    }).sort_values('importance', ascending=True)
    
    ax3.barh(feature_importance['feature'], feature_importance['importance'], 
             alpha=0.7, color='green')
    ax3.set_xlabel('Absolute Coefficient Value')
    ax3.set_title('Feature Importance (Current Model)')
    ax3.grid(True, alpha=0.3)
    
    # 4. Model version summary
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    summary_text = f"""
    📋 Model Summary
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    Current Version: {versions[-1]}
    
    Total Versions: {len(versions)}
    
    Baseline Accuracy: {pipeline.baseline_performance:.4f}
    Current Accuracy: {performances[-1]:.4f}
    
    Replay Buffer Size: {len(pipeline.replay_buffer)}
    
    Alert Thresholds:
      • Drift: p-value < {pipeline.drift_threshold}
      • Performance: drop > {pipeline.performance_threshold}
    
    Status: ✅ Active
    """
    
    ax4.text(0.1, 0.5, summary_text, fontsize=11, family='monospace',
             verticalalignment='center')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Dashboard created successfully!")

# Create dashboard
create_monitoring_dashboard(pipeline)

---
## Practice 7: Rollback Mechanism

### 🎯 Learning Objectives
- Implement model rollback functionality
- Test rollback scenarios
- Maintain model version history

### 📖 Key Concepts
**Rollback**: Reverting to a previous model version when new model underperforms

In [ ]:
# 7.1 Implement rollback mechanism
def demonstrate_rollback(pipeline):
    """
    Demonstrate model rollback to previous version
    """
    print("🔙 Demonstrating Rollback Mechanism")
    print("=" * 60)
    
    if len(pipeline.model_versions) < 2:
        print("⚠️ Not enough versions for rollback demonstration.")
        return
    
    # Current version info
    current_version = pipeline.model_versions[-1]
    previous_version = pipeline.model_versions[-2]
    
    print(f"Current version: {current_version['version']}")
    print(f"  Performance: {current_version['performance']:.4f}")
    print(f"\nPrevious version: {previous_version['version']}")
    print(f"  Performance: {previous_version['performance']:.4f}")
    
    # Simulate rollback
    print("\n🔄 Performing rollback...")
    pipeline.current_model = previous_version['model']
    
    print(f"✅ Rolled back to version {previous_version['version']}")
    print("=" * 60)
    
    # Visualize version history
    fig, ax = plt.subplots(figsize=(12, 6))
    
    versions = [v['version'] for v in pipeline.model_versions]
    performances = [v['performance'] for v in pipeline.model_versions]
    
    # Plot all versions
    ax.plot(versions, performances, marker='o', linewidth=2, 
            markersize=10, color='blue', alpha=0.5, label='All Versions')
    
    # Highlight current (rolled back) version
    ax.plot(versions[-2], performances[-2], marker='*', 
            markersize=20, color='green', label='Active (After Rollback)')
    
    # Highlight rolled back version
    ax.plot(versions[-1], performances[-1], marker='x', 
            markersize=15, color='red', label='Rolled Back')
    
    ax.axhline(y=pipeline.baseline_performance, color='green', 
               linestyle='--', alpha=0.5, label='Baseline')
    ax.set_xlabel('Model Version')
    ax.set_ylabel('Performance')
    ax.set_title('Model Version History with Rollback')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Rollback Scenarios:")
    print("   1. Performance degradation below threshold")
    print("   2. Safety issues detected in production")
    print("   3. Unexpected behavior or errors")
    print("   4. Regulatory compliance issues")

# Demonstrate rollback
demonstrate_rollback(pipeline)

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **Concept Drift Detection**: Using statistical tests (KS test) to detect data distribution changes
2. **Catastrophic Forgetting**: Understanding and measuring performance degradation on old tasks
3. **Memory Replay**: Implementing buffer-based strategy to preserve past knowledge
4. **Update Pipeline**: Building automated workflows for model updates
5. **Performance Monitoring**: Creating dashboards to track model health
6. **Rollback Mechanism**: Implementing safety mechanisms for production systems

### Key Insights:

✅ **Drift Detection is Critical**: Regular monitoring prevents silent model degradation

✅ **Memory Replay Works**: Simple buffer strategy reduces forgetting by >80%

✅ **Automation is Essential**: Pipelines ensure consistent, reliable updates

✅ **Safety First**: Always maintain rollback capability for production models

### Real-World Applications:

- **COVID-19 Response**: Models adapted to new variants while maintaining diagnostic accuracy
- **Seasonal Diseases**: Models update for flu season patterns while preserving year-round performance
- **Population Changes**: Adapting to demographic shifts in patient populations
- **Medical Guidelines**: Incorporating new treatment protocols and diagnostic criteria

### Next Steps:

1. Implement advanced strategies (EWC, Progressive Neural Networks)
2. Add A/B testing for gradual rollout
3. Integrate with MLOps tools (MLflow, DVC)
4. Develop regulatory documentation pipeline
5. Build multi-tier alert systems

---

## 📚 Additional Resources

**Papers**:
- "Continual Learning in Medical Imaging" (Nature Reviews)
- "Memory Replay for Continual Learning" (ICML)
- "Concept Drift Detection Methods" (ACM Survey)

**Tools**:
- Evidently AI: Drift detection and monitoring
- MLflow: Model versioning and tracking
- DVC: Data and model version control

**Regulatory**:
- FDA Predetermined Change Control Plan
- EU MDR Guidelines for AI/ML Medical Devices

---

### 🎓 Congratulations!

You've completed the Continuous Learning hands-on practice. You now have practical experience with:

- Building adaptive medical AI systems
- Detecting and responding to concept drift
- Preventing catastrophic forgetting
- Implementing production-ready update pipelines

**Keep learning and building robust, adaptive AI systems! 🚀**